#### Environment Setup

**(IMPORTANT)** Please always execute this code block before running any `pyomo` models.

In [ ]:
%%capture
import sys

if 'google.colab' in sys.modules:
    %pip install pyomo >/dev/null 2>/dev/null
    %pip install highspy >/dev/null 2>/dev/null

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

In [ ]:
model = pyo.ConcreteModel()

# sets
model.L = pyo.Set(initialize=['Wuksachi', 'Montecito'])
model.C = pyo.Set(initialize=['Beat', 'Aveo', 'Vento'])
model.A = pyo.Set(initialize=[
    'Sherman', 'Congress', 'Stargazing', 'Watchtower', 'MarbleFalls',
    'CrystalCave', 'MoroRock', 'BigTrees', 'AltaPeak'
])
model.D = pyo.Set(initialize=[1, 2, 3])  # Days

# params
lodging_cost = {'Wuksachi': 352, 'Montecito': 479}
breakfast_cost = {'Wuksachi': 40, 'Montecito': 60}
car_cost = {'Beat': 87, 'Aveo': 90, 'Vento': 93}
activity_duration = {
    'Sherman': 1.5, 'Congress': 1.0, 'Stargazing': 1.0, 'Watchtower': 4.5,
    'MarbleFalls': 3.5, 'CrystalCave': 2.0, 'MoroRock': 0.75,
    'BigTrees': 1.0, 'AltaPeak': 7.0
}
activity_cost = {
    'Sherman': 0, 'Congress': 0, 'Stargazing': 40, 'Watchtower': 0,
    'MarbleFalls': 0, 'CrystalCave': 40, 'MoroRock': 0,
    'BigTrees': 0, 'AltaPeak': 0
}
activity_time = {
    'Sherman': 'Afternoon', 'Congress': 'Afternoon', 'Stargazing': 'Night', 'Watchtower': 'Sunrise',
    'MarbleFalls': 'Sunrise', 'CrystalCave': 'Afternoon', 'MoroRock': 'Sunrise',
    'BigTrees': 'Afternoon', 'AltaPeak': 'Afternoon'
}
suggested_by = {
    'Sherman': 1, 'Congress': 2, 'Stargazing': 2, 'Watchtower': 1,
    'MarbleFalls': 2, 'CrystalCave': 2, 'MoroRock': 2,
    'BigTrees': 1, 'AltaPeak': 2
}
available_hours = {1: 5, 2: 11, 3: 7}
max_downtime = {1: 1, 2: 1, 3: 1}

activity_intensity = {
    'Sherman': 1,    # Easy
    'Congress': 2,   # Moderate
    'Stargazing': 1, # Easy
    'Watchtower': 2, # Moderate
    'MarbleFalls': 2,# Moderate
    'CrystalCave': 3,# Strenuous
    'MoroRock': 3,   # Strenuous
    'BigTrees': 1,   # Easy
    'AltaPeak': 3    # Strenuous
}

# pyo params
model.lodging_cost = pyo.Param(model.L, initialize=lodging_cost)
model.breakfast_cost = pyo.Param(model.L, initialize=breakfast_cost)
model.car_cost = pyo.Param(model.C, initialize=car_cost)
model.activity_cost = pyo.Param(model.A, initialize=activity_cost)
model.duration = pyo.Param(model.A, initialize=activity_duration)
model.time_of_day = pyo.Param(model.A, initialize=activity_time, within=pyo.Any)
model.suggester = pyo.Param(model.A, initialize=suggested_by)
model.available_hours = pyo.Param(model.D, initialize=available_hours)
model.max_downtime = pyo.Param(model.D, initialize=max_downtime)
model.intensity = pyo.Param(model.A, initialize=activity_intensity)

# decision var
model.use_lodge = pyo.Var(model.L, domain=pyo.Binary)
model.use_car = pyo.Var(model.C, domain=pyo.Binary)
model.schedule = pyo.Var(model.A, model.D, domain=pyo.Binary)
model.day_hours = pyo.Var(model.D, domain=pyo.NonNegativeReals)
model.fatigue_penalty = pyo.Var(model.D, domain=pyo.NonNegativeReals)
model.intensity_sum = pyo.Var(model.D, domain=pyo.NonNegativeReals)


In [ ]:
#constraints
# 1 one lodge and one car selected
model.one_lodge = pyo.Constraint(expr=sum(model.use_lodge[l] for l in model.L) == 1)
model.one_car = pyo.Constraint(expr=sum(model.use_car[c] for c in model.C) == 1)

# 2 Total activity hours per day
def calc_day_hours(m, d):
    return m.day_hours[d] == sum(m.schedule[a, d] * m.duration[a] for a in m.A)
model.calc_hours = pyo.Constraint(model.D, rule=calc_day_hours)

# 3 Daily time limit
model.daily_limit = pyo.ConstraintList()
for d in model.D:
    model.daily_limit.add(model.day_hours[d] <= model.available_hours[d])

# 4 Downtime constraint
model.downtime_limit = pyo.ConstraintList()
for d in model.D:
    model.downtime_limit.add(model.day_hours[d] >= model.available_hours[d] - model.max_downtime[d])

# 5 Each activity scheduled on at most one day
def one_day_per_activity(m, a):
    return sum(m.schedule[a, d] for d in m.D) <= 1
model.once_per_activity = pyo.Constraint(model.A, rule=one_day_per_activity)

# 6 No sunrise hikes on Day 1
model.sunrise_day1_forbidden = pyo.ConstraintList()
for a in model.A:
    if activity_time[a] == 'Sunrise':
        model.sunrise_day1_forbidden.add(model.schedule[a, 1] == 0)

# 7 No night activities on Day 3
model.night_restriction = pyo.Constraint(expr=model.schedule['Stargazing', 3] == 0)

# 8 Each day must include ≥1 activity suggested by other person
model.other_person_daily = pyo.ConstraintList()
for d in model.D:
    model.other_person_daily.add(sum(model.schedule[a, d] for a in model.A if suggested_by[a] == 2) >= 1)

# 9 Congress Trail implies Sherman Trail on same day
model.congress_requires_sherman = pyo.ConstraintList()
for d in model.D:
    model.congress_requires_sherman.add(model.schedule['Congress', d] <= model.schedule['Sherman', d])

# 10 No two strenuous hikes on consecutive days
model.no_consecutive_strenuous = pyo.ConstraintList()

for i in range(1, 3):
    model.no_consecutive_strenuous.add(
        sum(model.schedule[a, i] for a in model.A if model.intensity[a] == 3) +
        sum(model.schedule[a, i+1] for a in model.A if model.intensity[a] == 3)
        <= 1
    )

# 11 At least one activity must be scheduled
model.at_least_one_activity = pyo.Constraint(
    expr=sum(model.schedule[a, d] for a in model.A for d in model.D) >= 1
)

# Fatigue penalty is absolute deviation from target intensity 5

def calc_intensity_sum(m, d):
    return m.intensity_sum[d] == sum(m.schedule[a, d] * m.intensity[a] for a in m.A)
model.calc_intensity_sum = pyo.Constraint(model.D, rule=calc_intensity_sum)

def fatigue_abs_rule_1(m, d):
    return m.fatigue_penalty[d] >= model.intensity_sum[d] - 5

def fatigue_abs_rule_2(m, d):
    return m.fatigue_penalty[d] >= 5 - model.intensity_sum[d]

model.fatigue_abs_1 = pyo.Constraint(model.D, rule=fatigue_abs_rule_1)
model.fatigue_abs_2 = pyo.Constraint(model.D, rule=fatigue_abs_rule_2)


In [ ]:
#obj func
model.obj = pyo.Objective(
    expr=(
        sum(model.use_lodge[l] * model.lodging_cost[l] * 2 for l in model.L) +
        sum(model.use_car[c] * model.car_cost[c] for c in model.C) +
        sum(model.schedule[a, d] * model.activity_cost[a] for a in model.A for d in model.D) +
        10 * sum(model.fatigue_penalty[d] for d in model.D)  # linear penalty
    ),
    sense=pyo.minimize
)

In [ ]:
# solve
results = SOLVER.solve(model)

print("Selected Lodge:")
for l in model.L:
    if pyo.value(model.use_lodge[l]) > 0.5:
        print(f" - {l}")

print("Selected Car:")
for c in model.C:
    if pyo.value(model.use_car[c]) > 0.5:
        print(f" - {c}")

# ordering by
time_priority = {'Sunrise': 0, 'Morning': 1, 'Afternoon': 2, 'Night': 3}

print("Activity Schedule:")
for d in model.D:
    print(f"\nDay {d}:")

    sorted_activities = sorted(
        [a for a in model.A if pyo.value(model.schedule[a, d]) > 0.5],
        key=lambda a: time_priority[model.time_of_day[a]]
    )

    if not sorted_activities:
        print("  (No activities scheduled)")
    else:
        for a in sorted_activities:
            duration = pyo.value(model.duration[a])
            print(f"  - {a} ({model.time_of_day[a]}, {duration} hrs)")

print("Daily Hours:")
for d in model.D:
    print(f" Day {d}: {pyo.value(model.day_hours[d]):.2f} hrs")


print(f"Total Objective Value (Lodge + Bfast + Car Rental + Activities + Fatigue Penalty): ${pyo.value(model.obj):,.2f}")


Selected Lodge:
 - Wuksachi
Selected Car:
 - Beat
Activity Schedule:

Day 1:
  - CrystalCave (Afternoon, 2.0 hrs)
  - BigTrees (Afternoon, 1.0 hrs)
  - Stargazing (Night, 1.0 hrs)

Day 2:
  - Watchtower (Sunrise, 4.5 hrs)
  - MarbleFalls (Sunrise, 3.5 hrs)
  - Sherman (Afternoon, 1.5 hrs)
  - Congress (Afternoon, 1.0 hrs)

Day 3:
  - AltaPeak (Afternoon, 7.0 hrs)
Daily Hours:
 Day 1: 4.00 hrs
 Day 2: 10.50 hrs
 Day 3: 7.00 hrs
Total Objective Value (Lodge + Bfast + Car Rental + Activities + Fatigue Penalty): $911.00
